## Install Lib API

In [ ]:
! pip install https://dnaink.jfrog.io/artifactory/dna-ink-pypi/model-fkeywords/0.1.0/model_fkeywords-0.1.0-py3-none-any.whl

In [ ]:
! python -m spacy download pt_core_news_sm

## Import libs

In [1]:
import pandas as pd
from api_model.nlsuper import NlExtractorProcess

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', None)

/tmp/ipykernel_21041/3084512904.py:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Variables

##### Description of process
        whats_process = 'complete'
            return: process all pipeline
        whats_process = 'partial'
            return: findkeywords and process bigrams
        whats_process = 'only_keywords'
            return: findkeywords   

In [2]:
filename = 'LIBERACAO_MARGEM'
prefix = 'CSV'
prefix_sep = '|'
column_text = 'TRANSCRICAO'
whats_process = 'complete'
#whats_process = 'partial'
#whats_process = 'only_keywords'
id_database = 'ID'
type_find = 'fixo' # aproximado

In [3]:
text_finds = {
    'negatividade': ['a desejar', 'abrir uma reclamacao', 'absurd', 'aguento mais', 'boceta', 'bosta', 'brigaram', 'buceta', 'burocra', 'cansad', 'caralho', 'conflito', 'constrang', 'decepcao', 'decepcionad', 'quanta demora', 'que demora', 'tentando resolver', 'descontent', 'descrenca', 'descrente', 'desrespeit', 'fdp', 'filha da puta', 'filha de uma puta', 'filho da puta', 'filho de uma puta', 'frustracao', 'frustrad', 'humilhacao', 'humilhad', 'ignorad', 'ignoram', 'incompet', 'inferno', 'informa nada', 'injuria', 'ironia', 'ironic', 'irreponsaveis', 'irresponsabilidade', 'irresponsavel', 'ma vontade', 'mal a pior', 'mal atendid', 'mal educad', 'mal respondem', 'mal tratad', 'mau atendid', 'mau educad', 'mau respondem', 'mau tratad', 'merda', 'nao podemos arcar', 'nao posso arcar', 'nao sabe', 'ninguem resolve', 'ofenderam', 'ofendid', 'orrivel', 'pelo amor de deus', 'pessimo', 'pessimu', 'porra', 'poxa', 'pqp', 'puta que pariu', 'reclamacao', 'reclamar', 'ridicul', 'ruim', 'sabe nada', 'sem educacao', 'ta dificil', 'trata mal', 'trata mau', 'tratou mal', 'tratou mau', 'triste', 'vergonh', 'vagabund', 'raiva', 'detest', 'nao quero mais', 'odeio', 'safad', 'deus me livre', 'vergonh'],
    'reputacao': ['advogad', 'consumidor.gov', 'entrar com processo', 'facebook', 'instagram', 'judiciais', 'judicial', 'justica', 'meu direito', 'meus direito', 'ouvidoria', 'pequenas causas', 'processar voces', 'procon', 'reclame aqui', 'reclameaqui', 'twitter'],
    'rechamada': ['ainda nao', 'alguma posicao', 'ate agora', 'ate o momento', 'atraso', 'chamado aberto', 'consigo resolver', 'contato novamente', 'continuo sem acesso', 'de novo', 'demora', 'desde ontem', 'desde semana', 'diversas vezes', 'duas vezes', 'em andamento', 'entrei em contato', 'era pra ter', 'faz um mes', 'faz uma semana', 'ja abri', 'ja enviei', 'ja liguei', 'ja mandei', 'liguei pra central', 'mesmo erro', 'muitas vezes', 'nao chegou', 'nao e a primeira vez', 'ocorrencia aberta', 'posicionamento', 'segunda vez', 'sem sucesso', 'tentativa', 'terceira vez', 'todo dia', 'varias vezes', 'chamado aberto'],
    'satisfacao': ['adorei', 'adoro', 'ageis', 'agil', 'agilidade', 'amei', 'atendimento perfeito', 'bem atendid', 'carisma', 'diferenciad', 'educad', 'eficacia', 'eficas', 'eficaz', 'elogiar', 'elogio', 'excelente atendimento', 'feliz', 'gostei', 'impecavel', 'me ajudou', 'merito', 'motivacao', 'motivad', 'muito 10', 'muito bom', 'muito dez', 'nota 10', 'nota dez', 'otimo atendimento', 'parabenizar', 'parabens', 'pela ajuda', 'personalizad', 'preparad', 'prestativ', 'profissional', 'qualidade', 'rapidez', 'resolveu', 'satisfacao', 'satisfatori', 'satisfeit', 'solucionou', 'ate que enfim', 'ate que em fim'],
    'concorrencia': ['ifood', 'caju', 'vee', 'flash', 'bem', 'up', 'VR', 'ticket', 'alelo', 'swile'],
    'contencao': ['desculpa', 'desculpe', 'sinto muito', 'desculpe a demora', 'lamento', 'lamentamos', 'perdoa', 'perdao', 'pelo ocorrido', 'o ocorrido', 'fique tranquil'],
    'ocorrencia': ['chamado', 'ocorrencia', 'registrada com sucesso'],
    'direcionamento': ['central de atedimento', 'ligar na central', 'somente na central', 'sodexo.com', 'ligue na central'],
    'pedefacil': ['clique no', 'clique em', 'menu servicos', 'do menu', 'pede facil', 'clica no', 'clica em', 'diretamente no site', 'visualizar em', 'no menu', 'menu'],
    'ativo': ['analista pode entrar em contato', 'o analista entrara em contato', 'para te auxiliar na proposta', 'solicitei o contato', 'deixe seu contato']

          }

In [4]:
df = NlExtractorProcess.call_process(filename, prefix, prefix_sep,\
                                     column_text, whats_process,\
                                     text_finds, id_database, type_find)

read file /opt/dna/find-keywords/datalake/CSV


/tmp/ipykernel_21041/561833192.py:1: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  df = NlExtractorProcess.call_process(filename, prefix, prefix_sep,\


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67266 entries, 0 to 67265
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CPF           67266 non-null  int64  
 1   DURATION      67266 non-null  int64  
 2   ENDTIME       67266 non-null  object 
 3   EXTERNALID    67266 non-null  object 
 4   ID            67266 non-null  int64  
 5   STARTTIME     67266 non-null  object 
 6   MONOCHANNEL   137 non-null    object 
 7   SEQUENCIA     67213 non-null  float64
 8   CANAL         67213 non-null  object 
 9   TRANSCRICAO   67213 non-null  object 
 10  AGENT         67266 non-null  object 
 11  WORKGROUP     67266 non-null  object 
 12  CATEGORIES    67266 non-null  object 
 13  TOPICS        66977 non-null  object 
 14  RECHAMADAS    352 non-null    float64
 15  URA_LOG       67266 non-null  object 
 16  FILA          57233 non-null  object 
 17  VQ            31683 non-null  object 
 18  CALLTYPE      67266 non-nu

22/06/11 22:54:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


root
 |-- ID: string (nullable = true)
 |-- TRANSCRICAO: string (nullable = true)

None
remove null values of dataset


count rows after remove null values 18488
+-------+----------------+
|ID     |TRANSCRICAO     |
+-------+----------------+
|6837807|tres tres       |
|6837840|com             |
|6837840|sei             |
|6838011|alo             |
|6837807|bom isso alo alo|
+-------+----------------+
only showing top 5 rows

None
process bigrams and trigrams of column_text


+-------+----------------+-------------+---------------+----------------+
|ID     |TRANSCRICAO     |countent_word|countent_bigram|countent_trigram|
+-------+----------------+-------------+---------------+----------------+
|6837807|tres tres       |tres         |tres tres      |null            |
|6837840|com             |null         |null           |null            |
|6837840|sei             |sei          |null           |null            |
|6838011|alo             |alo          |null           |null            |
|6837807|bom isso alo alo|alo          |bom alo        |null            |
+-------+----------------+-------------+---------------+----------------+
only showing top 5 rows

dataframe with wordCloud ..None
convert to pandas again
dataframe old ['CPF', 'DURATION', 'ENDTIME', 'EXTERNALID', 'ID', 'STARTTIME', 'MONOCHANNEL', 'SEQUENCIA', 'CANAL', 'TRANSCRICAO', 'AGENT', 'WORKGROUP', 'CATEGORIES', 'TOPICS', 'RECHAMADAS', 'URA_LOG', 'FILA', 'VQ', 'CALLTYPE', 'CONTACTID', 'DATA_ARQUIVO

In [5]:
df.head()

,CPF,DURATION,ENDTIME,EXTERNALID,ID,STARTTIME,MONOCHANNEL,SEQUENCIA,CANAL,TRANSCRICAO_x,...,contencao,ocorrencia,direcionamento,pedefacil,ativo,numbers_words,TRANSCRICAO_y,countent_word,countent_bigram,countent_trigram
0,40361829,39,03MAR2022:11:47:38,DE59V9JKTH2GP6JKP00AEM2EPK029HK4,6837807,03MAR2022:11:46:59,NaN,0.0,LEFT,tres tres,...,,,,,,2,tres tres,tres,tres tres,None
1,40361829,39,03MAR2022:11:47:38,DE59V9JKTH2GP6JKP00AEM2EPK029HK4,6837807,03MAR2022:11:46:59,NaN,0.0,LEFT,tres tres,...,,,,,,2,bom isso alo alo,alo,bom alo,None
2,40361829,9,03MAR2022:11:50:10,DE59V9JKTH2GP6JKP00AEM2EPK029HLK,6837840,03MAR2022:11:50:01,NaN,0.0,RIGHT,com,...,,,,,,1,com,None,None,None
3,40361829,9,03MAR2022:11:50:10,DE59V9JKTH2GP6JKP00AEM2EPK029HLK,6837840,03MAR2022:11:50:01,NaN,0.0,RIGHT,com,...,,,,,,1,sei,sei,None,None
4,40361829,583,03MAR2022:12:01:33,DE59V9JKTH2GP6JKP00AEM2EPK029HNV,6838011,03MAR2022:11:51:50,NaN,0.0,LEFT,,...,,,,,,1,alo,alo,None,None
